In [1]:
import pandas as pd
import re
import numpy as np
data = pd.read_csv('GSAF5.csv', encoding='latin')

In [2]:
#1. check if the columns contain NaN5992 rows total, but both Unnamed #23 and #23 
#have majority of NaN values.
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [3]:
#2. DROP those two Unnamed columns with this piece of code:
drop_cols = list(null_cols[null_cols > 5000].index)
data = data.drop(drop_cols, axis=1)
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
dtype: object

In [4]:
#3.Print Case Number columns for analysis.
case_col=['Case Number', 'Case Number.1', 'Case Number.2']
data[case_col].head(3)

,Case Number,Case Number.1,Case Number.2
0,2016.09.18.c,2016.09.18.c,2016.09.18.c
1,2016.09.18.b,2016.09.18.b,2016.09.18.b
2,2016.09.18.a,2016.09.18.a,2016.09.18.a


In [5]:
#4. comparer les 3 colonnes. Not finish. Don't know how to attack this, as df.diff does not work, as values are strings and not int
Case=data['Case Number']
print(Case.head(2))

0    2016.09.18.c
1    2016.09.18.b
Name: Case Number, dtype: object


In [6]:
Case.equals(Case1)

NameError: name 'Case1' is not defined

In [32]:
Case.equals(Case2)

False

In [120]:
data['Match?'] = np.where(data['Case Number.1'] == data['Case Number'], 'True', 'False')  #create new column in df1 to check if prices match
print (data['Match?'].head(10))

0     True
1     True
2     True
3     True
4    False
5     True
6     True
7     True
8     True
9     True
Name: Match?, dtype: object


In [67]:
data['Match?2'] = np.where(data['Case Number.2'] == data['Case Number'], 'True', 'False')  #create new column in df1 to check if prices match


In [45]:
#6. After comparison or/and correction of values in Case Number. Drop Case1 and Case2. 
# In future need to do check with other columns - to understand which value is correct.
# Output 5992 rows × 20 columns
data1=data.drop(columns=['Case Number.1', 'Case Number.2'])


In [46]:
#7. Check which columns are full.
null_cols = data1.isnull().sum()
null_cols[null_cols ==0]

Case Number       0
Date              0
Year              0
Type              0
pdf               0
original order    0
Match?            0
Match?2           0
dtype: int64

In [90]:
#8. Date Serie object with only Date column to clean up.
DateSerie= pd.DataFrame(data, columns = ['Date'])
DateSerie.head(5000)

,Date
0,18-Sep-16
1,18-Sep-16
2,18-Sep-16
3,17-Sep-16
4,16-Sep-16
5,15-Sep-16
6,11-Sep-16
7,07-Sep-16
8,06-Sep-16
9,05-Sep-16


In [91]:
#9. Creating column with Date DD-MMM-YY format pattern [0-9]{1,2}-[A-Z]\w{2}-[0-9]{1,2}
import re
format1=DateSerie.Date.str.match('[0-9]{1,2}-[A-Z]\w{2}-[0-9]{1,2}')

ListTrue=[x for x in format1 if x == True]

print(len(ListTrue))
#4559 rows corresponding to the format DD-MMM-YY out of 5992 rows.
#next step is to check remainig 1433 rows.

4559


In [92]:
#Format2.Between YYYY & - YYYY [A-Z][a-z]+\s\d{1,4}\w\d{1,4}\s[-|&]\s\d* (Between)\s\d*\s&\s\d*'
format2=DateSerie.Date.str.match('\w+\s\d{1,4}\w\d{1,4}\s[-|&]\s\d*')
ListTrue2=[x for x in format2 if x == True]

print(len(ListTrue2))
# 2 rows which corresponds to "Between YYYY & YYYY"

1


In [93]:
#Format3. ANY char chain 'Reported / before' DD-MMM-YY pattern= [A-Z][a-z]+\s\d{1,2}-[A-Z]\w{2}-[0-9]{1,2}
format3=DateSerie.Date.str.match('[A-Z][a-z]+\s\d{1,2}-[A-Z]\w{2}-[0-9]{1,2}')
ListTrue3=[x for x in format3 if x == True]
print(len(ListTrue3))
# 462 rows which corresponds to "Anychar chain DD-MMM-YY"
# 971 rows remaining with other formats
#[A-Z][a-z]+\s\d*

462


In [96]:
#Format4. Before 1987
import datetime
format4=DateSerie.Date.str.match('(Before)\s\d*')
format4
ListTrue4=[ x for x in format4 if x == True]
print(len(ListTrue4))
# 65 rows which corresponds to "Anychar chain YYYY"
s=pd.Series(pd.to_datetime(ListTrue4, format="%d%b%y"))

65


ValueError: time data True does not match format '%d%b%y' (match)

In [79]:

import datetime

d = datetime.datetime.strptime("2013-1-25", '%Y-%m-%d')
print(datetime.date.strftime(d, "%m-%b-%y"))

01-Jan-13


In [60]:
#Format5. Early 1987
format5=df.Date.str.match('(Early)\s\d*')

ListTrue5=[x for x in format5 if x == True]
print(len(ListTrue5))
# 65 rows which corresponds to "Anychar chain YYYY"

13


In [100]:
Col_date_boolean={"format1": format1, "format2":format2, "format3":format3}
date_table=pd.DataFrame(Col_date_boolean)
date_table.head(3)
#not finished. Need to change the Report YYYY to YYYY
#

,format1,format2,format3
0,True,False,False
1,True,False,False
2,True,False,False


In [204]:
#df['Month'] = df['Date'].str.extract('([A-Z]\w{0,})', expand=True)

In [103]:
#df.head(100)

# supprimer les reported (les espaces : strip ) => 1

#format 1 : nrows = strftime (specifier format) dateutil (package) %d- %Y
#(aussi le cas avec les reported, une fois ces derniers supprimés)
#format 2 : nrows = 


In [105]:
print(set(data['Age']))

{nan, '44', '34', '69', '"middle-age"', 'mid-30s', '1', '12 or 13', '25 or 28', '53', '41', '46', '72', '21', '22', '35', '52', '61', '21, 34,24 & 35', '30s', '54', '49', 'F', '65', '77', '13 or 14', '57', '37, 67, 35, 27,  ? & 27', '78', '16', '3', '73', '7 or 8', '6', '21 or 26', '36', '17', '13 or 18', '34 & 19', '17 & 16', '7', 'adult', '86', '12', '62', '43', '21 & ?', '45', '56', '8 or 10', '66', '26', '23 & 20', '30', '8', '28 & 26', '33', '17 & 35', 'X', 'MAKE LINE GREEN', '18', '64', '13', 'A.M.', '32', '27', '33 & 26', '84', '9 & 12', '48', '68', '55', '9 months', '37', 'Both 11', '2½', '25 to 35', '60s', '50', '60', '15', '67', 'young', '20?', '? & 19', 'Teens', '39', '33 or 37', '40s', '5', '18 to 22', '71', '75', '74', 'teen', '33 & 37', 'Ca. 33', '18 or 20', '23 & 26', 'mid-20s', ' ', '51', '7      &    31', '30 & 32', '40', '42', '36 & 23', '87', 'Teen', '20', '28', '19', '10 or 12', '(adult)', '47', '31 or 33', '2 to 3 months', 'Elderly', '20s', '30 or 36', '32 & 30', '

In [106]:
# Adults age
#df.replace(r'^\.\.\..*', np.nan, regex=True)
data1['Age'] = data['Age'].str.replace({np.nan: 'adult': 30, '(adult)': 30, '"middle-age"':  30})
data1

SyntaxError: invalid syntax (<ipython-input-106-4aed73b19682>, line 3)

In [ ]:
data1['Age'] = data['Age'].str.replace('Elderly', '60')
data1['Age'] = data['Age'].str.replace('(adult)', '40')
data1['Age'] = data['Age'].str.replace('Teens', '15')
data1['Age'] = data['Age'].str.replace('young', '10')
#data1['Age'] = data['Age'].str.replace('\d+\s*[m|M]\w*', '1')
print(set(data1['Age']))

In [210]:
#cleaned some issues in the Column Age
data['Age'] = data['Age'].str.replace('\w', '')
print(set(data1['Age']))

{nan, '21 or 26', '50s', '65', '73', '13 or 18', '30', '28 & 26', '35', '18 or 20', '6½', '30 & 32', '34', '33 & 37', 'young', '75', 'mid-20s', '56', '17', '49', '68', '7', ' ', '23 & 20', '25 or 28', '23', '\xa0 ', '"young"', '36 & 26', '50', '?    &   14', '87', '28', '36', '10', '43', '9 months', '45', '20?', '86', '55', 'Elderly', '"middle-age"', '63', '25', '78', '16 to 18', '24', '61', 'Teen', '12 or 13', '81', '48', '26', '8 or 10', '40', '53', '5', 'mid-30s', '30s', '84', '59', '47', '10 or 12', '28, 23 & 30', '58', '32 & 30', '17 & 16', '29', '9 & 12', '42', '21 & ?', '13', '70', 'F', '33', '44', '(adult)', '66', '36 & 23', '20', '6', '7      &    31', '2½', '12', '3', '21, 34,24 & 35', '34 & 19', 'M', '30 or 36', '72', 'teen', '14', '18 months', '52', 'A.M.', '11', '17 & 35', '60s', '77', '>50', '21', 'Teens', '38', 'MAKE LINE GREEN', '37, 67, 35, 27,  ? & 27', 'Both 11', '39', '37', '33 or 37', '64', '62', '9', '22', '2 to 3 months', 'adult', '19', '20s', '71', '7 or 8', '16

In [117]:
data1['Type'] = data1['Type'].str.replace('Boating', 'Boat')

In [118]:
data1.to_csv('Result_Project1.csv', index=False)